# PROJECT OVERVIEW

## We aim to perform **portfolio analysis**, **visualization**, and **optimization**.
## Portfolio optimization is done with the objective of maximizing returns and reducing risks.

## This project has four parts:
### - Performing Data Visualization for One Stock
### - Performing Data Visualization for Multiple Stock, including analyzing correlations between stocks
### - Simulating the allocation of capital into a portfolio of stocks & analyzing the returns, risks, & Sharpe.
### - Optimizing the portfolio for the best Sharpe Ratio & exploring Markowitz' Efficient Frontier



# IMPORT LIBRARIES AND DATASETS

In [ ]:
# !pip install plotly==4.14.3

In [ ]:
# Import key librares and modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Patching deprecated aliases
np.bool = bool
np.int = int
np.float = float
import seaborn as sns

import plotly.io as pio
import plotly.express as px

import cufflinks as cf
cf.go_offline() # Enabling offline mode for interactive data visualization locally
# print(pio.renderers)

import yfinance as yf

# Import datetime module that comes pre-installed in Python 
# datetime offers classes that work with date & time information
import datetime as dt

In [ ]:
# Use Pandas to read stock data (the csv file is included in the course package) 
stock_df = pd.read_csv('Amazon.csv')
stock_df = stock_df.set_index('Date')
stock_df.head(15)

In [ ]:
# Count the number of missing values in "stock_df" Pandas DataFrame
stock_df.isnull().sum()

In [ ]:
# Obtain information about the Pandas DataFrame such as data types, memory utilization..etc
stock_df.info()

In [ ]:
# repeat the process for JPM
JPM_df = pd.read_csv("JPM.csv")
JPM_df = JPM_df.set_index('Date')
JPM_df['Daily Return'] = JPM_df['Adj Close'].pct_change(1)*100
JPM_df = JPM_df.fillna(0)

In [ ]:
JPM_df.info()

In [ ]:
JPM_df.isnull().sum()

# CALCULATE PERCENTAGE DAILY RETURNS

In [ ]:
# Calculate the percentage daily return
stock_df['Daily Return'] = stock_df['Adj Close'].pct_change(1) * 100
stock_df['Daily Return'].fillna(0, inplace=True)
stock_df

In [ ]:
# Use the describe() method to obtain a statistical summary about the data 
# Over the specified time period, the average adjusted close price for Amazon stock was $120.07 
# The maximum adjusted close price was $186.57
# The maximum volume of shares traded on one day were 311,346,000
stock_df.describe().round(2)

In [ ]:
JPM_df['Daily Returns'] = JPM_df['Adj Close'].pct_change(1)*100
JPM_df['Daily Returns'].fillna(0, inplace = True)
JPM_df

In [ ]:
JPM_df.describe()

# PERFORM DATA VISUALIZATION FOR A SINGLE STOCK: PART 1

In [ ]:
# Plot a Line Plot Using Plotly Express
fig = px.line(title = 'Amazon.com, Inc. (AMZN) Adjusted Closing Price [$]')
fig.add_scatter(x = stock_df.index, y = stock_df['Adj Close'], name = 'Adj Close')
fig.show(renderer="browser")

In [ ]:
# Define a function that performs interactive data visualization using Plotly Express
def plot_financial_data(df, title, renderer):
    
    fig = px.line(title = title)
    
    # For loop that plots all stock prices in the pandas dataframe df
    # Note that index starts with 1 because we want to skip the date column
    
    for i in df.columns:
        fig.add_scatter(x = df.index, y = df[i], name = i)
        fig.update_traces(line_width = 5)
        fig.update_layout({'plot_bgcolor': "white"})

    fig.show(renderer=renderer)

In [ ]:
# Plot High, Low, Open, Close and Adj Close
plot_financial_data(df = stock_df.drop(['Volume', 'Daily Return'], axis = 1), title = 'Amazon.com, Inc. (AMZN) Stock Price [$]', renderer='browser')

In [ ]:
# Plot trading volume
plot_financial_data(stock_df.iloc[:,[5]], 'Amazon.com, Inc. (AMZN) Trading Volume', 'browser')

In [ ]:
# Plot % Daily Returns
plot_financial_data(stock_df.iloc[:,[6]], 'Amazon.com, Inc. (AMZN) Percentage Daily Return [%]', 'notebook')

In [ ]:
s = stock_df['Daily Return'].describe()
plt.hist(stock_df['Daily Return'], bins=40)
x = np.arange(-15, 15, 0.01)

g = (1/np.sqrt(2*np.pi*s[2]*s[2]))*np.exp(-((x-s[1])**2)/(2*s[2]*s[2]))*s[0]
plt.plot(x, g)
plt.show()

In [ ]:
# Plot a Line Plot Using Plotly Express
fig = px.line(title = 'JPM')
for i in JPM_df.drop(['Volume', 'Daily Return'], axis=1).columns[1:]:
    fig.add_scatter(x = stock_df.index, y = JPM_df[i], name=i)
fig.show(renderer="notebook")

In [ ]:
fig = px.line(title='JPM vol')
fig.add_scatter(x=JPM_df.index, y=JPM_df['Volume'], name='Volume')
fig.show(renderer='notebook')

In [ ]:
fig = px.line()
fig.add_scatter(x=JPM_df.index, y=JPM_df['Adj Close'])
fig.show(renderer='notebook')

# PERFORM DATA VISUALIZATION FOR A SINGLE STOCK: PART 2

In [ ]:
# Define a function that classifies the returns based on the magnitude
# Feel free to change these numbers
def percentage_return_classifier(percentage_return):
    
    if percentage_return > -0.3 and percentage_return <= 0.3:
        return 'Insignificant Change'
    elif percentage_return > 0.3 and percentage_return <= 3:
        return 'Positive Change'
    elif percentage_return > -3 and percentage_return <= -0.3:
        return 'Negative Change'
    elif percentage_return > 3 and percentage_return <= 7:
        return 'Large Positive Change'
    elif percentage_return > -7 and percentage_return <= -3:
        return 'Large Negative Change'
    elif percentage_return > 7:
        return 'Bull Run'
    elif percentage_return <= -7:
        return 'Bear Sell Off'

In [ ]:
# Apply the function to the "Daily Return" Column and place the result in "Trend" column
stock_df['Trend'] = stock_df['Daily Return'].apply(percentage_return_classifier)
stock_df

In [ ]:
# Count distinct values in the Trend column
trend_summary_AMZN = stock_df['Trend'].value_counts()
trend_summary_AMZN

In [ ]:
# Plot a pie chart using Matplotlib Library
# plt.figure(figsize = (8, 8))
trend_summary_AMZN.plot(kind = 'pie', y = 'Trend');

JPM_df['Trend'] = JPM_df['Daily Return'].apply(percentage_return_classifier)
ts_JPM = JPM_df['Trend'].value_counts()
plt.figure()
ts_JPM.plot(kind="pie")
plt.show()

# PERFORM DATA VISUALIZATION FOR A SINGLE STOCK: PART 3

In [ ]:
# Plot Candlestick figure using Cufflinks QuantFig module
cf.go_offline()             # Use offline mode
# cf.set_config_file(world_readable=True, theme='white')  # Set default theme
figure = cf.QuantFig(stock_df, title = 'Amazon.com, Inc. (AMZN) Candlestick Chart', name = 'AMZN')
# figure = cf.QuantFig(stock_df)
#figure.add_sma(periods =[14, 21], column = 'Close', color = ['magenta', 'green'])
figure.iplot(theme = 'white', up_color = 'green', down_color = 'red')

In [ ]:
JPM_df['SMA_20']=JPM_df['Close'].rolling(window=20).mean()
JPM_df['std_dev']=JPM_df['Close'].rolling(window=20).std()
JPM_df['B-band Upper'] = JPM_df['SMA_20']+2*JPM_df['std_dev']
JPM_df['B-band Lower'] = JPM_df['SMA_20']-2*JPM_df['std_dev']
JPM_df.drop('std_dev', axis=1)

In [ ]:
fig = cf.QuantFig(JPM_df)
fig.add_sma([30, 100], column='Open',color=['magenta', 'blue'])
fig.add_bollinger_bands(periods=20, boll_std=2, fill=True, column='Close', color='orange')
fig.add_bollinger_bands(periods=20, boll_std=1, fill=True, column='Close', color='red')
fig.iplot(theme='white', up_color='green', down_color='red')

# PERFORM DATA VISUALIZATION FOR MULTIPLE STOCKS

### We will focus our analysis on the following U.S. stocks :-

**AMZN:** Amazon Inc. - Multinational tech company focusing on e-commerce, cloud computing, and artificial intelligence

**JPM:** JPMorgan Chase and Co. - Multinational investment bank and financial services holding company

**META:** Meta Platforms, formerly named Facebook Inc. - META owns Facebook, Instagram, and WhatsApp

**PG:** Procter and Gamble (P&G) - Multinational consumer goods corporation

**GOOG:** Google (Alphabet Inc.) - Multinational company that focuses on search engine tech, e-commerce, Cloud and AI 

**CAT:** Caterpillar - World's largest construction-equipment manufacturer

**PFE:** Pfizer Inc. - Multinational pharmaceutical and biotechnology corporation

**EXC:** Exelon - An American Fortune 100 energy company 

**DE:** Deere & Company (John Deere) - Manufactures agricultural machinery and heavy equipment

**JNJ:** Johnson & Johnson - A multinational corporation that develops medical devices and pharmaceuticals


In [ ]:
close_price_df = pd.read_csv('stock_prices.csv')
close_price_df = close_price_df.set_index('Date')
close_price_df

In [ ]:
# The objective of this code cell is to calculate the percentage daily return
daily_returns_df = close_price_df.iloc[:, :].pct_change() * 100
daily_returns_df.replace(np.nan, 0, inplace = True)
daily_returns_df

In [ ]:
# Plot closing prices using plotly Express. Note that we used the same pre-defined function "plot_financial_data"
plot_financial_data(close_price_df, 'Adjusted Closing Prices [$]', 'notebook')

In [ ]:
# Plot the stocks daily returns
plot_financial_data(daily_returns_df, 'Percentage Daily Returns [%]', 'browser')

In [ ]:
# Plot histograms for stocks daily returns using plotly express
# Compare META to JNJ daily returns histograms
fig = px.histogram(daily_returns_df)
#fig = px.histogram(daily_returns_df['AMZN'], nbins=20000)
fig.update_layout({'plot_bgcolor': "white"})

In [ ]:
# Plot a heatmap showing the correlations between daily returns
# Strong positive correlations between Catterpillar and John Deere - both into heavy equipment and machinery
# META and Google - both into Tech and Cloud Computing
plt.figure(figsize = (10, 8))
sns.heatmap(daily_returns_df.corr(), annot = True);

In [ ]:
daily_returns_df

In [ ]:
# Plot the Pairplot between stocks daily returns
sns.pairplot(daily_returns_df);

In [ ]:
# Function to scale stock prices based on their initial starting price
# The objective of this function is to set all prices to start at a value of 1 
def price_scaling(raw_prices_df):
    scaled_prices_df = raw_prices_df.copy()
    for i in raw_prices_df.columns:
          scaled_prices_df[i] = raw_prices_df[i]/raw_prices_df[i][0]
    return scaled_prices_df

In [ ]:
price_scaling(close_price_df)

# DEFINE A FUNCTION THAT GENERATES RANDOM PORTFOLIO WEIGHTS

In [ ]:
# Let's create an array that holds random portfolio weights
# Note that portfolio weights must add up to 1 
import random

def generate_portfolio_weights(n):
    weights = []
    for i in range(n):
        weights.append(random.random())
    # let's make the sum of all weights add up to 1
    weights = weights/np.sum(weights)
    return weights

In [ ]:
w = generate_portfolio_weights(10)
print(w)
print(sum(w))

# PERFORM ASSET ALLOCATION & CALCULATE PORTFOLIO DAILY VALUE/RETURN

In [ ]:
# Let's display "close_price_df" Pandas DataFrame
close_price_df

In [ ]:
# Scale stock prices using the "price_scaling" function that we defined earlier (make all stock values start at 1)
portfolio_df = close_price_df.copy()
scaled_df = price_scaling(portfolio_df)
scaled_df

In [ ]:
# Use enumerate() method to obtain the stock names along with a counter "i" (0, 1, 2, 3,..etc.)
# This counter "i" will be used as an index to access elements in the "weights" list
initial_investment = 1000000
for i, stock in enumerate(scaled_df.columns):
    portfolio_df[stock] = weights[i] * scaled_df[stock]  * initial_investment
portfolio_df.round(1)

In [ ]:
# Assume that we have $1,000,000 that we would like to invest in one or more of the selected stocks
# Let's create a function that receives the following arguments: 
      # (1) Stocks closing prices
      # (2) Random weights 
      # (3) Initial investment amount
# The function will return a DataFrame that contains the following:
      # (1) Daily value (position) of each individual stock over the specified time period
      # (2) Total daily value of the portfolio 
      # (3) Percentage daily return 

def asset_allocation(df, weights, initial_investment):
    portfolio_df = df.copy()

    # Scale stock prices using the "price_scaling" function that we defined earlier (Make them all start at 1)
    scaled_df = price_scaling(df)
  
    for i, stock in enumerate(scaled_df.columns):
        portfolio_df[stock] = scaled_df[stock] * weights[i] * initial_investment

    # Sum up all values and place the result in a new column titled "portfolio value [$]" 
    # Note that we excluded the date column from this calculation
    portfolio_df['Portfolio Value [$]'] = portfolio_df.sum(axis = 1, numeric_only = True)
            
    # Calculate the portfolio percentage daily return and replace NaNs with zeros
    portfolio_df['Portfolio Daily Return [%]'] = portfolio_df['Portfolio Value [$]'].pct_change(1) * 100 
    portfolio_df.replace(np.nan, 0, inplace = True)
    
    return portfolio_df

In [ ]:
# Now let's put this code in a function and generate random weights
# Let's obtain the number of stocks under consideration (note that we ignored the "Date" column) 
n = len(close_price_df.columns)

# Let's generate random weights 
print('Number of stocks under consideration = {}'.format(n))
weights = generate_portfolio_weights(n)
print('Portfolio weights = {}'.format(weights))
print(sum(weights))
# Let's test out the "asset_allocation" function
portfolio_df = asset_allocation(close_price_df, weights, 1000000)
portfolio_df.round(2)

In [ ]:
# Plot the portfolio percentage daily return
plot_financial_data(portfolio_df[['Portfolio Daily Return [%]']], 'Portfolio Percentage Daily Return [%]', renderer='notebook')

# Plot each stock position in our portfolio over time
# This graph shows how our initial investment in each individual stock grows over time
plot_financial_data(portfolio_df.drop(['Portfolio Value [$]', 'Portfolio Daily Return [%]'], axis = 1), 'Portfolio positions [$]', renderer='notebook')

# Plot the total daily value of the portfolio (sum of all positions)
plot_financial_data(portfolio_df[['Portfolio Value [$]']], 'Total Portfolio Value [$]', renderer='notebook')

# DEFINE THE "SIMULATION" FUNCTION THAT PERFORMS ASSET ALLOCATION, AND CALCULATES KEY PORTFOLIO METRICS

In [ ]:
# Let's define the simulation engine function 
# The function receives: 
    # (1) portfolio weights
    # (2) initial investment amount
# The function performs asset allocation and calculates portfolio statistical metrics including Sharpe ratio
# The function returns: 
    # (1) Expected portfolio return 
    # (2) Expected volatility 
    # (3) Sharpe ratio 
    # (4) Return on investment 
    # (5) Final portfolio value in dollars


def simulation_engine(weights, initial_investment, rf):
    # Perform asset allocation using the random weights (sent as arguments to the function)
    portfolio_df = asset_allocation(close_price_df, weights, initial_investment)
  
    # Calculate the return on the investment 
    # Return on investment is calculated using the last final value of the portfolio compared to its initial value
    return_on_investment = ((portfolio_df['Portfolio Value [$]'][-1:] - 
                             portfolio_df['Portfolio Value [$]'][0])/ 
                             portfolio_df['Portfolio Value [$]'][0]) * 100
  
    
    cagr = (return_on_investment.values[0]*0.01)**(251/(len(portfolio_df.index)-1))

    # Daily change of every stock in the portfolio (Note that we dropped the date, portfolio daily worth and daily % returns) 
    portfolio_daily_return_df = portfolio_df.drop(columns = ['Portfolio Value [$]', 'Portfolio Daily Return [%]'])
    portfolio_daily_return_df = portfolio_daily_return_df.pct_change(1) 
  
    # Portfolio Expected Return formula
    expected_portfolio_return = np.sum(weights * portfolio_daily_return_df.mean() ) * 252
  
    # Portfolio volatility (risk) formula
    # The risk of an asset is measured using the standard deviation which indicates the dispertion away from the mean
    # The risk of a portfolio is not a simple sum of the risks of the individual assets within the portfolio
    # Portfolio risk must consider correlations between assets within the portfolio which is indicated by the covariance 
    # The covariance determines the relationship between the movements of two random variables
    # When two stocks move together, they have a positive covariance when they move inversely, the have a negative covariance 

    covariance = portfolio_daily_return_df.cov() * 252 
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

    # Check out the chart for the 10-years U.S. treasury at https://ycharts.com/indicators/10_year_treasury_rate
    # rf = 0.03 # Try to set the risk free rate of return to 1% (assumption)

    # Calculate Sharpe ratio
    sharpe_ratio = (expected_portfolio_return - rf)/expected_volatility 
    return expected_portfolio_return, expected_volatility, sharpe_ratio, portfolio_df['Portfolio Value [$]'][-1:].values[0], return_on_investment.values[0], cagr
  

In [ ]:
# Let's test out the "simulation_engine" function and print out statistical metrics
# Define the initial investment amount
initial_investment = 1000000
portfolio_metrics = simulation_engine(weights, initial_investment, 0.03)

In [ ]:
print('Expected Portfolio Annual Return = {:.2f}%'.format(portfolio_metrics[0] * 100))
print('Portfolio Standard Deviation (Volatility) = {:.2f}%'.format(portfolio_metrics[1] * 100))
print('Sharpe Ratio = {:.2f}'.format(portfolio_metrics[2]))
print('Portfolio Final Value = ${:.2f}'.format(portfolio_metrics[3]))
print('Return on Investment = {:.2f}%'.format(portfolio_metrics[4]))
print(f"CAGR = {(portfolio_metrics[5]-1)*100:.2f}%")

# RUN MONTE CARLO SIMULATIONS 

In [ ]:
# Set the number of simulation runs
sim_runs = 10000
initial_investment = 1000000

# Placeholder to store all weights
weights_runs = np.zeros((sim_runs, n))

# Placeholder to store all Sharpe ratios
sharpe_ratio_runs = np.zeros(sim_runs)

# Placeholder to store all expected returns
expected_portfolio_returns_runs = np.zeros(sim_runs)

# Placeholder to store all volatility values
volatility_runs = np.zeros(sim_runs)

# Placeholder to store all returns on investment
return_on_investment_runs = np.zeros(sim_runs)

# Placeholder to store all final portfolio values
final_value_runs = np.zeros(sim_runs)

# Placeholder to store all cagr values
cagr_runs = np.zeros(sim_runs)

for i in range(sim_runs):
    # Generate random weights 
    weights = generate_portfolio_weights(n)
    # Store the weights
    weights_runs[i,:] = weights
    
    # Call "simulation_engine" function and store Sharpe ratio, return and volatility
    # Note that asset allocation is performed using the "asset_allocation" function  
    expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i], final_value_runs[i], return_on_investment_runs[i], cagr_runs[i] = simulation_engine(weights, initial_investment, 0.03)


# PERFORM PORTFOLIO OPTIMIZATION 

In [ ]:
# List all Sharpe ratios generated from the simulation
sharpe_ratio_runs

In [ ]:
# Return the index of the maximum Sharpe ratio (Best simulation run) 
sharpe_ratio_runs.argmax()

In [ ]:
# Return the maximum Sharpe ratio value
sharpe_ratio_runs.max()

In [ ]:
weights_runs

In [ ]:
# Obtain the portfolio weights that correspond to the maximum Sharpe ratio (Golden set of weights!)
weights_runs[sharpe_ratio_runs.argmax(), :]

In [ ]:
# Return Sharpe ratio, volatility corresponding to the best weights allocation (maximum Sharpe ratio)
optimal_portfolio_return, optimal_volatility, optimal_sharpe_ratio, highest_final_value, optimal_return_on_investment, optimal_cagr = simulation_engine(weights_runs[sharpe_ratio_runs.argmax(), :], initial_investment, 0.03)

In [ ]:
print('Best Portfolio Metrics Based on {} Monte Carlo Simulation Runs:'.format(sim_runs))
print('  - Portfolio Expected Annual Return = {:.02f}%'.format(optimal_portfolio_return * 100))
print('  - Portfolio Standard Deviation (Volatility) = {:.02f}%'.format(optimal_volatility * 100))
print('  - Sharpe Ratio = {:.02f}'.format(optimal_sharpe_ratio))
print('  - Final Value = ${:.02f}'.format(highest_final_value))
print('  - Return on Investment = {:.02f}%'.format(optimal_return_on_investment))

In [ ]:
# Create a DataFrame that contains volatility, return, and Sharpe ratio for all simualation runs
sim_out_df = pd.DataFrame({'Volatility': volatility_runs.tolist(), 'Portfolio_Return': expected_portfolio_returns_runs.tolist(), 'Sharpe_Ratio': sharpe_ratio_runs.tolist() })
sim_out_df

In [ ]:
# Let's highlight the point with the highest Sharpe ratio
fig = px.scatter(sim_out_df, x = 'Volatility', y = 'Portfolio_Return', color = 'Sharpe_Ratio', size = 'Sharpe_Ratio', hover_data = ['Sharpe_Ratio'] )
fig.add_trace(go.Scatter(x = [optimal_volatility], y = [optimal_portfolio_return], mode = 'markers', name = 'Optimal Point', marker = dict(size=[40], color = 'red')))
fig.update_layout(coloraxis_colorbar = dict(y = 0.7, dtick = 5))
fig.update_layout({'plot_bgcolor': "white"})
fig.show()

# APPENDIX

In [ ]:
# Code to obtain stock data using Pandas Datareader
# Select the start/end dates and ticker symbols
# Data is saved in a stock_data.csv file
tickers = ['AMZN', 'JPM', 'META', 'PG', 'GOOGL', 'CAT', 'PFE', 'EXC', 'DE', 'JNJ'] 
tickers = ['AMZN'] 

!pip install pandas_datareader
!pip install yfinance
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

# Indicate the start and end dates
start = dt.datetime(2018, 1, 1)
end = dt.datetime.now()

df = pdr.get_data_yahoo(tickers, start = start, end = end)
print(df)
df.to_csv('stock_data.csv')

In [ ]:
tickers = ['AMZN', 'JPM', 'META', 'PG', 'GOOG', 'CAT', 'PFE', 'EXC', 'DE', 'JNJ']
data = yf.download(tickers, start='2014-01-01', end='2024-06-01', auto_adjust=True)
data['Close'].to_csv('stock_prices.csv')

Run this to check the version of plotly and cufflinks:

!pip show plotly cufflinks

Plotly should be version 4.14.3 or older. The newer one conflicts with cufflinks. 